# Data Collection
In this notebook, we join all the preprossed data from different sources to a new table. 

In [2]:
import pandas as pd 
from sodapy import Socrata
import unidecode
import numpy as np

## 1. Create base
The codes from each municipality are taken from the national department of statistics [DANE](https://geoportal.dane.gov.co/servicios/descarga-y-metadatos/#gsc.tab=0)

#### TO DO: 
* short description of what does DIPLOVA mean 
* check what municipalities joined etc

In [3]:
# read table 
muni=pd.read_excel("DANE\DIVIPOLA_Municipios.xlsx",skiprows=10)

# get only the information on municipalities
muni=muni[muni['Unnamed: 4']=='Municipio']

# rename columns 
rename_dict={'Código ':'depcode','Código .1':'muncode','Nombre.1': 'munname'}
muni=muni.rename(columns=rename_dict)

# get only relevant columns 
muni=muni[['depcode','Nombre','muncode','munname']]

# code muncode as integer
muni['muncode']=[int(code) for code in muni['muncode']]

In [4]:
muni.head(5)

,depcode,Nombre,muncode,munname
0,05,ANTIOQUIA,5001,MEDELLÍN
1,05,ANTIOQUIA,5002,ABEJORRAL
2,05,ANTIOQUIA,5004,ABRIAQUÍ
3,05,ANTIOQUIA,5021,ALEJANDRÍA
4,05,ANTIOQUIA,5030,AMAGÁ


In [5]:
# create a data frame for the years 
years=pd.DataFrame(range(1988,2022),columns=['year'])

# create base from crossproduct of municipality codes and years 
index = pd.DataFrame(index=pd.MultiIndex.from_product([years['year'], muni['muncode']], names=['year', 'muncode'])).reset_index()

# join the other informations
base=index.merge(muni,on=['muncode'])

In [6]:
base.head(5)

,year,muncode,depcode,Nombre,munname
0,1988,5001,05,ANTIOQUIA,MEDELLÍN
1,1989,5001,05,ANTIOQUIA,MEDELLÍN
2,1990,5001,05,ANTIOQUIA,MEDELLÍN
3,1991,5001,05,ANTIOQUIA,MEDELLÍN
4,1992,5001,05,ANTIOQUIA,MEDELLÍN


## 2. Join Panel level Data 

### 2.1 SIEVCAC 
The [SIEVAC](https://micrositios.centrodememoriahistorica.gov.co/observatorio/sievcac/) project has many data tables which contain information on the events. 
* The information has been aggregated to extract the annual number of clashes, goverment attacks, guerrilla attacks, paramilitary attacks, post demobilizytion groups attacks, guerrilla massacres, paramilitary massacres, post demobilization groups massacres, guerrilla kidnappings, paramilitary kidnappings.
* Since, some events might be inlcuded in several tables, the inofrmation has been joined using the [MELLT](https://journals.sagepub.com/doi/full/10.1177/0022002718777050) method 

### TO DO: 
* Write down which criteria were used (taxonomy, spazial fuzziness etc)

### DONE:
* Revise "final" MELTT implementaion and import dataset --> aggregate the data 

In [7]:
# read data
sievcac=pd.read_csv('SIEVCAC\SIEVCAC_data.csv')

#drop extra column
sievcac=sievcac.drop('Unnamed: 0', axis=1)

In [8]:
base=base.merge(sievcac,how='left',on=['year','muncode'],validate='1:1')
# fill empty values with 0 
base=base.fillna(0)

In [9]:
base

,year,muncode,depcode,Nombre,munname,clashes,govattacks,guerrattacks,parattacks,posdattacks,parmass,guerrmass,posdmass,parsec,guerrsec,posdsec,n_parsec,n_guerrsec,n_posdsec,causalities
0,1988,5001,05,ANTIOQUIA,MEDELLÍN,9.0,1.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,30.0
1,1989,5001,05,ANTIOQUIA,MEDELLÍN,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,6.0,0.0,30.0
2,1990,5001,05,ANTIOQUIA,MEDELLÍN,18.0,10.0,0.0,0.0,0.0,2.0,3.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,191.0
3,1991,5001,05,ANTIOQUIA,MEDELLÍN,9.0,5.0,4.0,0.0,0.0,1.0,8.0,0.0,0.0,7.0,0.0,0.0,7.0,0.0,175.0
4,1992,5001,05,ANTIOQUIA,MEDELLÍN,22.0,1.0,4.0,0.0,0.0,2.0,3.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,149.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37497,2017,99773,99,VICHADA,CUMARIBO,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
37498,2018,99773,99,VICHADA,CUMARIBO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37499,2019,99773,99,VICHADA,CUMARIBO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0,0.0,0.0
37500,2020,99773,99,VICHADA,CUMARIBO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 2.2 HRDAG
The [HRDAG](https://hrdag.org/jep-cev-colombia/) did a joint project with the special peace jurisdiction and the national statistics deparment in which human right violations were documented. Here is interesting that they used advanced imputation techniques to join the information. I will use the information they have on kidnappings to check the reobusteness of the results. The data is highly correlated with the measures from SIEVCAC. 
* The coverage is only between 1990 and 2018

### TO DO: 
* See if I can do pivot and renaming directly in r. 
* Get demobilization kidnappings (see DATA OBSERVATIONS/IDEA...)

In [10]:
# read table (this steps should be done in R)
hrdag_sec=pd.read_csv('HRDAG\HRDAG_secuestros.csv')
hrdag_sec['p_str']=hrdag_sec['p_str'].fillna('nan')
hrdag_sec_w=hrdag_sec.pivot(index=['yy_hecho', 'muni_code_hecho'], columns='p_str',values='observed')

# fill nan with 0 since it means no kidnappings from said group
hrdag_sec_w=hrdag_sec_w.fillna(0).reset_index()
hrdag_sec_w['guersec_HRDAG']=hrdag_sec_w['GUE-ELN']+hrdag_sec_w['GUE-FARC']
hrdag_sec_w=hrdag_sec_w.rename(columns={'yy_hecho':'year','muni_code_hecho':'muncode', 'PARA':'parsec_HRDAG', 'GUE-OTRO':'posdsec_HRDAG'})
hrdag_sec_w=hrdag_sec_w.drop(columns=['nan','EST','GUE-ELN','GUE-FARC','OTRO','multiple'])


In [11]:
hrdag_sec_w.head(5)

p_str,year,muncode,posdsec_HRDAG,parsec_HRDAG,guersec_HRDAG
0,1990,5001.0,1.0,1.0,2.0
1,1990,5031.0,0.0,0.0,1.0
2,1990,5045.0,0.0,1.0,2.0
3,1990,5051.0,1.0,0.0,0.0
4,1990,5088.0,0.0,0.0,0.0


In [12]:
# join data to base 
base=base.merge(hrdag_sec_w,how='left',on=['year','muncode'])

In [13]:
#since some municipalities just never expirienced kidnappings 
for varname in ['posdsec_HRDAG','parsec_HRDAG','guersec_HRDAG']:
    base[f'{varname}']=base[f'{varname}'].fillna(0)

In [14]:
base.head(5)

,year,muncode,depcode,Nombre,munname,clashes,govattacks,guerrattacks,parattacks,posdattacks,...,parsec,guerrsec,posdsec,n_parsec,n_guerrsec,n_posdsec,causalities,posdsec_HRDAG,parsec_HRDAG,guersec_HRDAG
0,1988,5001,05,ANTIOQUIA,MEDELLÍN,9.0,1.0,7.0,0.0,0.0,...,0.0,3.0,0.0,0.0,3.0,0.0,30.0,0.0,0.0,0.0
1,1989,5001,05,ANTIOQUIA,MEDELLÍN,9.0,0.0,0.0,0.0,0.0,...,0.0,6.0,0.0,0.0,6.0,0.0,30.0,0.0,0.0,0.0
2,1990,5001,05,ANTIOQUIA,MEDELLÍN,18.0,10.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,2.0,0.0,191.0,1.0,1.0,2.0
3,1991,5001,05,ANTIOQUIA,MEDELLÍN,9.0,5.0,4.0,0.0,0.0,...,0.0,7.0,0.0,0.0,7.0,0.0,175.0,2.0,0.0,4.0
4,1992,5001,05,ANTIOQUIA,MEDELLÍN,22.0,1.0,4.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,149.0,0.0,2.0,1.0


### 2.3 CEDE
Form andes university


In [15]:
# load data 
cede_panel=pd.read_csv('CEDE\CEDE_data.csv')

#drop weird column
    # y_cap_regalias (can still be changed) cede_panel.drop(columns=['y_cap_regalias','CPI','frac'],inplace=True)
cede_panel=cede_panel.drop(['Unnamed: 0','y_cap_regalias'], axis=1)

In [16]:
base=base.merge(cede_panel,how='left',on=['year','muncode'],validate='1:1')

In [17]:
base.head(4)

,year,muncode,depcode,Nombre,munname,clashes,govattacks,guerrattacks,parattacks,posdattacks,...,n_posdsec,causalities,posdsec_HRDAG,parsec_HRDAG,guersec_HRDAG,ac_cafe,p_cafe,H_coca,pobl_tot,lpop
0,1988,5001,05,ANTIOQUIA,MEDELLÍN,9.0,1.0,7.0,0.0,0.0,...,0.0,30.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,1989,5001,05,ANTIOQUIA,MEDELLÍN,9.0,0.0,0.0,0.0,0.0,...,0.0,30.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,1990,5001,05,ANTIOQUIA,MEDELLÍN,18.0,10.0,0.0,0.0,0.0,...,0.0,191.0,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN
3,1991,5001,05,ANTIOQUIA,MEDELLÍN,9.0,5.0,4.0,0.0,0.0,...,0.0,175.0,2.0,0.0,4.0,NaN,NaN,NaN,NaN,NaN


### 2.4 Rainfall and temperature

In [18]:
climate=pd.read_csv('CHELSA\climate_muncode.csv')
climate.drop(columns='Unnamed: 0',inplace=True)

In [19]:
climate.head(3)

,muncode,_prcmean,year,_tempmean
0,5002,32340.527031,2006,NaN
1,5002,32785.080252,2007,2905.964846
2,5002,36580.726190,2008,2902.933333


In [20]:
base=base.merge(climate,how='left',on=['muncode','year'],validate='1:1')

### 2.5 SGR
Oil production from 2010 to 2021

In [21]:
oilp=pd.read_csv('SGR\oil_prod_revenue.csv')
oilp.drop(columns='Unnamed: 0',inplace=True)

In [22]:
base=base.merge(oilp,how='left',on=['muncode','year'],validate='1:1')

In [23]:
base.head(3)

,year,muncode,depcode,Nombre,munname,clashes,govattacks,guerrattacks,parattacks,posdattacks,...,guersec_HRDAG,ac_cafe,p_cafe,H_coca,pobl_tot,lpop,_prcmean,_tempmean,regalias_cop,prod_gravable_bls_kpc
0,1988,5001,05,ANTIOQUIA,MEDELLÍN,9.0,1.0,7.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1989,5001,05,ANTIOQUIA,MEDELLÍN,9.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1990,5001,05,ANTIOQUIA,MEDELLÍN,18.0,10.0,0.0,0.0,0.0,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 2.6 ANH 
Oil production data at municipality level for 1989 to 2010.

In [24]:
oilp2=pd.read_csv('AHN\oil_1989_2010.csv',index_col=0)

In [25]:
base=base.merge(oilp2,how='left',on=['muncode','year'],validate='1:1')
base['oil_prod']=base['oil_prod'].fillna(0)


In [26]:
base['prod_gravable_bls_kpc']=base['prod_gravable_bls_kpc'].fillna(0)

In [27]:
# for the year of overlaping dta both variables are highly correlated 
base[base['year']==2010][['oil_prod','prod_gravable_bls_kpc']].corr()

,oil_prod,prod_gravable_bls_kpc
oil_prod,1.000000,0.980157
prod_gravable_bls_kpc,0.980157,1.000000


In [28]:
# create new variable for production 
base['oil_production']=base['oil_prod']+base['prod_gravable_bls_kpc']
# for observations in 2010 take the average
base['oil_production']=[oil if year!=2010 else oil/2 for oil,year in zip(base['oil_production'],base['year'])]

In [29]:
print(base[base['year']==2010][['oil_prod','prod_gravable_bls_kpc','oil_production']].sum())

oil_prod                 286.608448
prod_gravable_bls_kpc    289.612524
oil_production           288.110486
dtype: float64


### 2.7 Terridata
Panel of territprial data. Here used to get capital revenue.
I used thousends of pesos 

#### TO DO 
* Ask Anke if 0.33 correlation is good enough for filling in data 

In [30]:
revenues=pd.read_csv('TerriData/revenue_2006.csv',index_col=0)

In [31]:
base=base.merge(revenues,how='left',on=['muncode','year'],validate='1:1')

In [32]:
base['caprev_2006']=base['caprev_2006_percapita']*base['pobl_tot']
base['lcaprev']=np.log(base['caprev_2006']/1000+0.001)

In [33]:
base

,year,muncode,depcode,Nombre,munname,clashes,govattacks,guerrattacks,parattacks,posdattacks,...,_prcmean,_tempmean,regalias_cop,prod_gravable_bls_kpc,oil_prod,oil_production,CapitalRevenue,caprev_2006_percapita,caprev_2006,lcaprev
0,1988,5001,05,ANTIOQUIA,MEDELLÍN,9.0,1.0,7.0,0.0,0.0,...,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN
1,1989,5001,05,ANTIOQUIA,MEDELLÍN,9.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN
2,1990,5001,05,ANTIOQUIA,MEDELLÍN,18.0,10.0,0.0,0.0,0.0,...,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN
3,1991,5001,05,ANTIOQUIA,MEDELLÍN,9.0,5.0,4.0,0.0,0.0,...,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN
4,1992,5001,05,ANTIOQUIA,MEDELLÍN,22.0,1.0,4.0,0.0,0.0,...,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37497,2017,99773,99,VICHADA,CUMARIBO,0.0,3.0,0.0,0.0,0.0,...,22748.722359,2989.794851,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,-6.907755
37498,2018,99773,99,VICHADA,CUMARIBO,0.0,0.0,0.0,0.0,0.0,...,28305.467340,2988.030349,0.0,0.0,0.0,0.0,7129897.0,4.351315e+06,3.163014e+11,19.572206
37499,2019,99773,99,VICHADA,CUMARIBO,0.0,0.0,0.0,0.0,0.0,...,NaN,2991.914059,0.0,0.0,0.0,0.0,2614053.0,1.541042e+06,1.174213e+11,18.581279
37500,2020,99773,99,VICHADA,CUMARIBO,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,0.0,3184810.0,1.831248e+06,1.444177e+11,18.788221


## 3. Join yearly data

### 3.1 Top 3 Coffee Exporters 

#### TO DO: 
* Check if data dube and Vargas includes vietnam
* If it does, check if data is similar 
* If it is, add data from dube and vargas 
* see wich unit is used in wits

In [34]:
# load data 
coffeexports=pd.read_csv('WITS\Top4CoffeeExports.csv')

# remove colombia 
top3exp=coffeexports[coffeexports['ReporterISO3']!='COL']

# aggregate exports by year
top3exp=top3exp[['Year','NetWeight in KGM']].groupby('Year').sum().reset_index()


# rename column for merging 
top3exp.rename(columns={'Year':'year','NetWeight in KGM':'top3cof'},inplace=True)


In [35]:
# merge to base 
base=base.merge(top3exp,how='left',on=['year'],validate='m:1')

In [36]:
base.head(5)

,year,muncode,depcode,Nombre,munname,clashes,govattacks,guerrattacks,parattacks,posdattacks,...,_tempmean,regalias_cop,prod_gravable_bls_kpc,oil_prod,oil_production,CapitalRevenue,caprev_2006_percapita,caprev_2006,lcaprev,top3cof
0,1988,5001,05,ANTIOQUIA,MEDELLÍN,9.0,1.0,7.0,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,1989,5001,05,ANTIOQUIA,MEDELLÍN,9.0,0.0,0.0,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,1.308774e+09
2,1990,5001,05,ANTIOQUIA,MEDELLÍN,18.0,10.0,0.0,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,1.270728e+09
3,1991,5001,05,ANTIOQUIA,MEDELLÍN,9.0,5.0,4.0,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,1.467416e+09
4,1992,5001,05,ANTIOQUIA,MEDELLÍN,22.0,1.0,4.0,0.0,0.0,...,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,1.259467e+09


### 3.1 Coffee and Oil prices

In [37]:
# load data 
prices=pd.read_csv('Prices\poil_pcoffee_2006COP.csv', usecols=['year','pcoffee2006','poil2006'] )
# merge to base 
base=base.merge(prices,how='left',on=['year'],validate='m:1')

In [38]:
base.head(3)

,year,muncode,depcode,Nombre,munname,clashes,govattacks,guerrattacks,parattacks,posdattacks,...,prod_gravable_bls_kpc,oil_prod,oil_production,CapitalRevenue,caprev_2006_percapita,caprev_2006,lcaprev,top3cof,pcoffee2006,poil2006
0,1988,5001,05,ANTIOQUIA,MEDELLÍN,9.0,1.0,7.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,709790.986715,64102.847584
1,1989,5001,05,ANTIOQUIA,MEDELLÍN,9.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,1.308774e+09,712126.733642,79626.343583
2,1990,5001,05,ANTIOQUIA,MEDELLÍN,18.0,10.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,1.270728e+09,680456.303960,105496.209859


# Add variables from Dube & Vargas (for now)


In [39]:
# read Dube & Vargas 
DV=pd.read_stata("Dube & Vargas/origmun_violence_commodities.dta", convert_categoricals=False)

# Create help variables 

### Coca production dummy
* If coca was produced in 1999

In [40]:
# create subset of the year 1999
y1999=base[base['year']==1999]

# generate dummy
y1999['coca99']=[1 if coca>0 else 0 for coca in y1999['H_coca']]

# keep only relevant variables 
y1999=y1999[['muncode','coca99']]

C:\Users\anasa\AppData\Local\Temp\ipykernel_32860\3910755344.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y1999['coca99']=[1 if coca>0 else 0 for coca in y1999['H_coca']]


In [41]:
# merge to base 
base=base.merge(y1999,how='left',on=['muncode'],validate='m:1')

In [42]:
base

,year,muncode,depcode,Nombre,munname,clashes,govattacks,guerrattacks,parattacks,posdattacks,...,oil_prod,oil_production,CapitalRevenue,caprev_2006_percapita,caprev_2006,lcaprev,top3cof,pcoffee2006,poil2006,coca99
0,1988,5001,05,ANTIOQUIA,MEDELLÍN,9.0,1.0,7.0,0.0,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,709790.986715,64102.847584,0
1,1989,5001,05,ANTIOQUIA,MEDELLÍN,9.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,1.308774e+09,712126.733642,79626.343583,0
2,1990,5001,05,ANTIOQUIA,MEDELLÍN,18.0,10.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,1.270728e+09,680456.303960,105496.209859,0
3,1991,5001,05,ANTIOQUIA,MEDELLÍN,9.0,5.0,4.0,0.0,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,1.467416e+09,634592.062209,85210.960679,0
4,1992,5001,05,ANTIOQUIA,MEDELLÍN,22.0,1.0,4.0,0.0,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,1.259467e+09,485647.559162,70169.310523,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37497,2017,99773,99,VICHADA,CUMARIBO,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000e+00,0.000000e+00,-6.907755,3.082158e+09,515489.063582,100662.256171,0
37498,2018,99773,99,VICHADA,CUMARIBO,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,7129897.0,4.351315e+06,3.163014e+11,19.572206,2.961198e+09,452290.152717,128685.990829,0
37499,2019,99773,99,VICHADA,CUMARIBO,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2614053.0,1.541042e+06,1.174213e+11,18.581279,3.975100e+09,464232.614321,124394.093262,0
37500,2020,99773,99,VICHADA,CUMARIBO,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3184810.0,1.831248e+06,1.444177e+11,18.788221,3.956256e+09,602700.176491,89106.821150,0


In [ ]:
base.to_csv('Preprocessed data\BaseConflict.csv')

In [44]:
base.columns

Index(['year', 'muncode', 'depcode', 'Nombre', 'munname', 'clashes',
       'govattacks', 'guerrattacks', 'parattacks', 'posdattacks', 'parmass',
       'guerrmass', 'posdmass', 'parsec', 'guerrsec', 'posdsec', 'n_parsec',
       'n_guerrsec', 'n_posdsec', 'causalities', 'posdsec_HRDAG',
       'parsec_HRDAG', 'guersec_HRDAG', 'ac_cafe', 'p_cafe', 'H_coca',
       'pobl_tot', 'lpop', '_prcmean', '_tempmean', 'regalias_cop',
       'prod_gravable_bls_kpc', 'oil_prod', 'oil_production', 'CapitalRevenue',
       'caprev_2006_percapita', 'caprev_2006', 'lcaprev', 'top3cof',
       'pcoffee2006', 'poil2006', 'coca99'],
      dtype='object')